In [1]:
import pytz
import os


DEFAULT_TZ = pytz.FixedOffset(540)  # GMT+09:00; Asia/Seoul

PATH_DATA = 'data/D'
PATH_ESM = os.path.join(PATH_DATA, 'EsmResponse.csv')
PATH_PARTICIPANT = os.path.join(PATH_DATA, 'UserInfo.csv')
PATH_SENSOR = os.path.join(PATH_DATA, 'Sensor')

PATH_INTERMEDIATE = os.path.join('/home/user/Collab/Data_Processing_D1/data/intermediate')
PATH_SAVE = '/home/user/Collab/MT/logo/3_clusters/data_MT'

DATA_TYPES = {
    'Acceleration': 'ACC',
    'AmbientLight': 'AML',
    'Calorie': 'CAL',
    'Distance': 'DST',
    'EDA': 'EDA',
    'HR': 'HRT',
    'RRI': 'RRI',
    'SkinTemperature': 'SKT',
    'StepCount': 'STP',
    'UltraViolet': 'ULV',
    'ActivityEvent': 'ACE',
    'ActivityTransition': 'ACT',
    'AppUsageEvent': 'APP',
    'BatteryEvent': 'BAT',
    'CallEvent': 'CAE',
    'Connectivity': 'CON',
    'DataTraffic': 'DAT',
    'InstalledApp': 'INS',
    'Location': 'LOC',
    'MediaEvent': 'MED',
    'MessageEvent': 'MSG',
    'WiFi': 'WIF',
    'ScreenEvent': 'SCR',
    'RingerModeEvent': 'RNG',
    'ChargeEvent': 'CHG',
    'PowerSaveEvent': 'PWS',
    'OnOffEvent': 'ONF'
}

In [2]:
import pandas as pd
import numpy as np
import scipy.stats as st
import cloudpickle
import ray
from datetime import datetime
from contextlib import contextmanager
import warnings
import time


def load(path: str):
    with open(path, mode='rb') as f:
        return cloudpickle.load(f)

    
def dump(obj, path: str):
    with open(path, mode='wb') as f:
        cloudpickle.dump(obj, f)
        
    
def log(msg: any):
    print('[{}] {}'.format(datetime.now().strftime('%y-%m-%d %H:%M:%S'), msg))


def summary(x):
    x = np.asarray(x)
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')

        n = len(x)
        # Here, uppercase np.dtype.kind corresponds to non-numeric data.
        # Also, we view the boolean data as dichotomous categorical data.
        if x.dtype.kind.isupper() or x.dtype.kind == 'b': 
            cnt = pd.Series(x).value_counts(dropna=False)
            card = len(cnt)
            cnt = cnt[:20]                
            cnt_str = ', '.join([f'{u}:{c}' for u, c in zip(cnt.index, cnt)])
            if card > 30:
                cnt_str = f'{cnt_str}, ...'
            return {
                'n': n,
                'cardinality': card,
                'value_count': cnt_str
            }
        else: 
            x_nan = x[np.isnan(x)]
            x_norm = x[~np.isnan(x)]
            
            tot = np.sum(x_norm)
            m = np.mean(x_norm)
            me = np.median(x_norm)
            s = np.std(x_norm, ddof=1)
            l, u = np.min(x_norm), np.max(x)
            conf_l, conf_u = st.t.interval(0.95, len(x_norm) - 1, loc=m, scale=st.sem(x_norm))
            n_nan = len(x_nan)
            
            return {
                'n': n,
                'sum': tot,
                'mean': m,
                'SD': s,
                'med': me,
                'range': (l, u),
                'conf.': (conf_l, conf_u),
                'nan_count': n_nan
            }


@contextmanager
def on_ray(*args, **kwargs):
    try:
        if ray.is_initialized():
            ray.shutdown()
        ray.init(*args, **kwargs)
        yield None
    finally:
        ray.shutdown()

transform = {
    'GAME': 'ENTER',
    'GAME_TRIVIA': 'ENTER',
    'GAME_CASINO': 'ENTER',
    'GAME-ACTION': 'ENTER',
    'GAME_SPORTS': 'ENTER',
    'GAME_PUZZLE': 'ENTER',
    'GAME_SIMULATION': 'ENTER',
    'GAME_STRATEGY': 'ENTER',
    'GAME_ROLE_PLAYING': 'ENTER',
    'GAME_ACTION': 'ENTER',
    'GAME_ARCADE': 'ENTER',
    'GAME_RACING': 'ENTER',
    'GAME_CASUAL': 'ENTER',
    'GAME_MUSIC': 'ENTER',
    'GAME_CARD': 'ENTER',
    'GAME_ADVENTURE': 'ENTER',
    'GAME_BOARD': 'ENTER',
    'GAME_EDUCATIONAL': 'ENTER',
    'GAME_RACING': 'ENTER',
    'PHOTOGRAPHY': 'ENTER',
    'ENTERTAINMENT': 'ENTER',
    'SPORTS': 'ENTER',
    'MUSIC_AND_AUDIO': 'ENTER',
    'COMICS': 'ENTER',
    'VIDEO_PLAYERS_AND_EDITORS': 'ENTER',
    'VIDEO_PLAYERS': 'ENTER',
    'ART_AND_DESIGN': 'ENTER',
    'TRAVEL_AND_LOCAL': 'INFO',
    'FOOD_AND_DRINK': 'INFO',
    'NEWS_AND_MAGAZINES': 'INFO',
    'MAPS_AND_NAVIGATION': 'INFO',
    'WEATHER': 'INFO',
    'HOUSE_AND_HOME': 'INFO',
    'BOOKS_AND_REFERENCE': 'INFO',
    'SHOPPING': 'INFO',
    'LIBRARIES_AND_DEMO': 'INFO',
    'BEAUTY': 'INFO',
    'AUTO_AND_VEHICLES': 'INFO',
    'LIFESTYLE': 'INFO',
    'PERSONALIZATION': 'SYSTEM',
    'TOOLS': 'SYSTEM',
    'COMMUNICATION': 'SOCIAL',
    'SOCIAL': 'SOCIAL',
    'DATING': 'SOCIAL',
    'PARENTING':'SOCIAL',
    'FINANCE': 'WORK',
    'BUSINESS': 'WORK',
    'PRODUCTIVITY': 'WORK',
    'EDUCATION': 'WORK',
    'HEALTH_AND_FITNESS': 'HEALTH',
    'MEDICAL': 'HEALTH',
    'SYSTEM': 'SYSTEM',
    'MISC': 'SYSTEM', # ABC logger
     None: 'UNKNOWN',
    'UNKNOWN':'UNKNOWN'
}

In [3]:
p = os.path.join(PATH_INTERMEDIATE, 'stress-fixed.pkl')
X, y, groups, t, datetimes = load(p)

In [4]:
PARTICIPANTS = pd.read_csv(os.path.join(PATH_INTERMEDIATE, 'PARTICIPANT_INFO.csv'),index_col = 'pcode')
PINFO = PARTICIPANTS.assign(
    BFI_OPN=lambda x: x['openness'],
    BFI_CON=lambda x: x['conscientiousness'],
    BFI_NEU=lambda x: x['neuroticism'],
    BFI_EXT=lambda x: x['extraversion'],
    BFI_AGR=lambda x: x['agreeableness'],
)[[
    'BFI_OPN', 'BFI_CON', 'BFI_NEU', 'BFI_EXT', 'BFI_AGR'
]]
PINFO = pd.get_dummies(PINFO, prefix_sep='=', dtype=bool)

In [5]:
LABELS_PROC = pd.read_csv(os.path.join(PATH_INTERMEDIATE, 'LABELS_PROC.csv'), index_col=['pcode','timestamp'],parse_dates=True)

In [6]:
_df =LABELS_PROC
_df.reset_index(level='timestamp', inplace=True)
print('First timestamp:', _df['timestamp'].min())
print('Last timestamp:', _df['timestamp'].max())

First timestamp: 2019-04-30 10:03:28+09:00
Last timestamp: 2019-05-22 22:02:03+09:00


In [7]:
time_ranges = _df.groupby('pcode')['timestamp'].agg(['min', 'max'])

In [8]:
list_pid = set(LABELS_PROC.index.get_level_values('pcode').values)

In [9]:
PINFO_valid = PINFO.loc[PINFO.index.isin(list_pid)]

In [10]:
#Divide the features into different categories
feat_current = X.loc[:,[('#VAL' in str(x)) or ('ESM#LastLabel' in str(x)) for x in X.keys()]]  
feat_dsc = X.loc[:,[('#DSC' in str(x))  for x in X.keys()]]  
feat_yesterday = X.loc[:,[('Yesterday' in str(x))  for x in X.keys()]]  
feat_today = X.loc[:,[('Today' in str(x))  for x in X.keys()]]  
feat_sleep = X.loc[:,[('Sleep' in str(x))  for x in X.keys()]]  
feat_time = X.loc[:,[('Time' in str(x))  for x in X.keys()]]  
feat_pif = X.loc[:,[('PIF' in str(x))  for x in X.keys()]]  
feat_ImmediatePast = X.loc[:,[('ImmediatePast_15' in str(x))  for x in X.keys()]]
#Divide the time window features into sensor/past stress label
feat_current_sensor = X.loc[:,[('#VAL' in str(x))  for x in X.keys()]]  
feat_current_ESM = X.loc[:,[('ESM#LastLabel' in str(x)) for x in X.keys()]]  
feat_ImmediatePast_sensor = feat_ImmediatePast.loc[:,[('ESM' not in str(x)) for x in feat_ImmediatePast.keys()]]  
feat_ImmediatePast_ESM = feat_ImmediatePast.loc[:,[('ESM'  in str(x)) for x in feat_ImmediatePast.keys()]]  
feat_today_sensor = feat_today.loc[:,[('ESM' not in str(x))  for x in feat_today.keys()]]  
feat_today_ESM = feat_today.loc[:,[('ESM'  in str(x)) for x in feat_today.keys()]]  
feat_yesterday_sensor = feat_yesterday.loc[:,[('ESM' not in str(x)) for x in feat_yesterday.keys()]]  
feat_yesterday_ESM = feat_yesterday.loc[:,[('ESM'  in str(x)) for x in feat_yesterday.keys()]]
#Prepare the final feature set
feat_baseline = pd.concat([ feat_time,feat_dsc,feat_current_sensor, feat_ImmediatePast_sensor],axis=1)
feat_final = pd.concat([feat_baseline ], axis=1)
X = feat_final

In [11]:
groups

array(['P01', 'P01', 'P01', ..., 'P80', 'P80', 'P80'], dtype='<U3')

In [12]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, precision_score, recall_score

skf = StratifiedKFold(n_splits=5)

# Lists to collect the metrics for each user
all_accs = []
all_aucs = []
all_f1s = []
all_f1p = []
all_precisions = []
all_recalls = []

# Loop through each user group
for user in PINFO_valid.index:
    print(f"Processing user: {user}")
    user_mask = groups == user
    X_user, y_user = X[user_mask], np.array(y)[user_mask]
    group_user = np.array(groups)[user_mask]
    
    user_accs = []
    user_aucs = []
    user_f1s = []
    user_f1p = []
    user_precisions = []
    user_recalls = []
    
    for train_idx, test_idx in skf.split(X_user, y_user):
        X_train, X_test = X_user.iloc[train_idx], X_user.iloc[test_idx]
        y_train, y_test = y_user[train_idx], y_user[test_idx]

        dtrain = xgb.DMatrix(X_train, label=y_train)
        dtest = xgb.DMatrix(X_test, label=y_test)
        
        # XGBoost parameters
        param = {
            "early_stopping_rounds": 200,
            "reg_alpha": 0,
            "colsample_bytree": 1,
            "colsample_bylevel": 1,
            "scale_pos_weight": 1,
            "learning_rate": 0.3,
            "nthread": 10,
            "min_child_weight": 1,
            "n_estimators": 1000,
            "subsample": 1,
            "reg_lambda": 1,
            "seed": 27,
            "objective": 'binary:logistic',
            "max_depth": 6,
            "gamma": 0,
            'eval_metric': 'auc',
            'silent': 1,
            'tree_method': 'exact',
            'debug': 0,
            'use_task_gain_self': 0,
            'when_task_split': 1,
            'how_task_split': 0,
            'min_task_gain': 0.0,
            'task_gain_margin': 0.0,
            'max_neg_sample_ratio': 0.4,
            'which_task_value': 2,
            'baseline_alpha': 1.0,
            'baseline_lambda': 1.0,
        }
        evallist = [(dtrain, 'train'), (dtest, 'test')]
        bst = xgb.train(param, dtrain, num_boost_round=1000, early_stopping_rounds=200, evals=evallist)

        y_real = dtest.get_label()
        y_score = bst.predict(dtest, ntree_limit=bst.best_ntree_limit)

        # Predict binary outcomes instead of probabilities
        y_pred = [1 if score >= 0.5 else 0 for score in y_score]

        # Calculate metrics
        # Calculate metrics
        user_accs.append(accuracy_score(y_real, y_pred))

        if len(np.unique(y_real)) > 1:
            user_aucs.append(roc_auc_score(y_real, y_score))
        else:
            user_aucs.append(np.nan)  # NaN indicates that ROC AUC couldn't be calculated

        user_f1s.append(f1_score(y_real, y_pred, average='macro'))
        user_precisions.append(precision_score(y_real, y_pred, average='macro'))
        user_recalls.append(recall_score(y_real, y_pred, average='macro'))
        user_f1p.append(f1_score(y_real, y_pred, pos_label=1))

        
    all_accs.extend(user_accs)
    all_aucs.extend(user_aucs)
    all_f1s.extend(user_f1s)
    all_f1p.extend(user_f1p)
    all_precisions.extend(user_precisions)
    all_recalls.extend(user_recalls)
    
    print(f"User: {user}, Avg. Accuracy: {np.mean(user_accs):.4f}, Avg. AUC: {np.nanmean(user_aucs):.4f}, Avg. F1: {np.mean(user_f1s):.4f}")

# Overall metrics for all users
print(f"\nOverall Metrics:")
print(f"Average Accuracy: {np.mean(all_accs):.4f}")
print(f"Average AUC: {np.nanmean(all_aucs):.4f}")
print(f"Average Macro F1 Score: {np.mean(all_f1s):.4f}")
print(f"Average Macro Precision: {np.mean(all_precisions):.4f}")
print(f"Average Macro Recall: {np.mean(all_recalls):.4f}")

Processing user: P01
[13:55:53] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1667849527992/work/src/learner.cc:767: 
Parameters: { "baseline_alpha", "baseline_lambda", "debug", "early_stopping_rounds", "how_task_split", "max_neg_sample_ratio", "min_task_gain", "n_estimators", "silent", "task_gain_margin", "use_task_gain_self", "when_task_split", "which_task_value" } are not used.

[0]	train-auc:0.89844	test-auc:0.20000
[1]	train-auc:0.91667	test-auc:0.23333
[2]	train-auc:0.98438	test-auc:0.26667
[3]	train-auc:1.00000	test-auc:0.26667
[4]	train-auc:1.00000	test-auc:0.26667
[5]	train-auc:1.00000	test-auc:0.10000
[6]	train-auc:1.00000	test-auc:0.16667
[7]	train-auc:1.00000	test-auc:0.16667
[8]	train-auc:1.00000	test-auc:0.16667
[9]	train-auc:1.00000	test-auc:0.06667
[10]	train-auc:1.00000	test-auc:0.06667
[11]	train-auc:1.00000	test-auc:0.06667
[12]	train-auc:1.00000	test-auc:0.06667
[13]	train-auc:1.00000	test-auc:0.06667
[14]	train-auc:1.00000	test-auc:0.06667
[15]	

[193]	train-auc:1.00000	test-auc:0.13333
[194]	train-auc:1.00000	test-auc:0.13333
[195]	train-auc:1.00000	test-auc:0.13333
[196]	train-auc:1.00000	test-auc:0.13333
[197]	train-auc:1.00000	test-auc:0.13333
[198]	train-auc:1.00000	test-auc:0.13333
[199]	train-auc:1.00000	test-auc:0.13333
[200]	train-auc:1.00000	test-auc:0.13333
[201]	train-auc:1.00000	test-auc:0.13333


/home/user/miniconda3/envs/sci-data/lib/python3.9/site-packages/xgboost/core.py:122: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  warnings.warn(


[13:56:01] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1667849527992/work/src/learner.cc:767: 
Parameters: { "baseline_alpha", "baseline_lambda", "debug", "early_stopping_rounds", "how_task_split", "max_neg_sample_ratio", "min_task_gain", "n_estimators", "silent", "task_gain_margin", "use_task_gain_self", "when_task_split", "which_task_value" } are not used.

[0]	train-auc:0.91063	test-auc:0.50000
[1]	train-auc:0.92271	test-auc:0.33333
[2]	train-auc:1.00000	test-auc:0.33333
[3]	train-auc:1.00000	test-auc:0.37500
[4]	train-auc:1.00000	test-auc:0.25000
[5]	train-auc:1.00000	test-auc:0.25000
[6]	train-auc:1.00000	test-auc:0.25000
[7]	train-auc:1.00000	test-auc:0.25000
[8]	train-auc:1.00000	test-auc:0.25000
[9]	train-auc:1.00000	test-auc:0.25000
[10]	train-auc:1.00000	test-auc:0.33333
[11]	train-auc:1.00000	test-auc:0.33333
[12]	train-auc:1.00000	test-auc:0.25000
[13]	train-auc:1.00000	test-auc:0.25000
[14]	train-auc:1.00000	test-auc:0.25000
[15]	train-auc:1.00000	tes

[194]	train-auc:1.00000	test-auc:0.25000
[195]	train-auc:1.00000	test-auc:0.25000
[196]	train-auc:1.00000	test-auc:0.25000
[197]	train-auc:1.00000	test-auc:0.25000
[198]	train-auc:1.00000	test-auc:0.25000
[199]	train-auc:1.00000	test-auc:0.25000
[200]	train-auc:1.00000	test-auc:0.25000


...

[193]	train-auc:1.00000	test-auc:0.50000
[194]	train-auc:1.00000	test-auc:0.50000
[195]	train-auc:1.00000	test-auc:0.50000
[196]	train-auc:1.00000	test-auc:0.50000
[197]	train-auc:1.00000	test-auc:0.50000
[198]	train-auc:1.00000	test-auc:0.50000
[199]	train-auc:1.00000	test-auc:0.50000
[200]	train-auc:1.00000	test-auc:0.50000
[201]	train-auc:1.00000	test-auc:0.50000
[202]	train-auc:1.00000	test-auc:0.50000
[203]	train-auc:1.00000	test-auc:0.50000
[204]	train-auc:1.00000	test-auc:0.50000
[205]	train-auc:1.00000	test-auc:0.50000
[206]	train-auc:1.00000	test-auc:0.50000
[207]	train-auc:1.00000	test-auc:0.50000
[208]	train-auc:1.00000	test-auc:0.50000
[209]	train-auc:1.00000	test-auc:0.50000
[210]	train-auc:1.00000	test-auc:0.50000
[211]	train-auc:1.00000	test-auc:0.50000
[212]	train-auc:1.00000	test-auc:0.50000
[213]	train-auc:1.00000	test-auc:0.50000
[214]	train-auc:1.00000	test-auc:0.50000
[215]	train-auc:1.00000	test-auc:0.50000
[216]	train-auc:1.00000	test-auc:0.50000
[217]	train-auc:

/home/user/miniconda3/envs/sci-data/lib/python3.9/site-packages/xgboost/core.py:122: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  warnings.warn(


[14:29:49] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1667849527992/work/src/learner.cc:767: 
Parameters: { "baseline_alpha", "baseline_lambda", "debug", "early_stopping_rounds", "how_task_split", "max_neg_sample_ratio", "min_task_gain", "n_estimators", "silent", "task_gain_margin", "use_task_gain_self", "when_task_split", "which_task_value" } are not used.

[0]	train-auc:0.97869	test-auc:0.55000
[1]	train-auc:0.98295	test-auc:0.55000
[2]	train-auc:1.00000	test-auc:0.42500
[3]	train-auc:1.00000	test-auc:0.30000
[4]	train-auc:1.00000	test-auc:0.30000
[5]	train-auc:1.00000	test-auc:0.40000
[6]	train-auc:1.00000	test-auc:0.40000
[7]	train-auc:1.00000	test-auc:0.40000
[8]	train-auc:1.00000	test-auc:0.45000
[9]	train-auc:1.00000	test-auc:0.45000
[10]	train-auc:1.00000	test-auc:0.45000
[11]	train-auc:1.00000	test-auc:0.40000
[12]	train-auc:1.00000	test-auc:0.40000
[13]	train-auc:1.00000	test-auc:0.35000
[14]	train-auc:1.00000	test-auc:0.30000
[15]	train-auc:1.00000	tes

[194]	train-auc:1.00000	test-auc:0.35000
[195]	train-auc:1.00000	test-auc:0.35000
[196]	train-auc:1.00000	test-auc:0.35000
[197]	train-auc:1.00000	test-auc:0.35000
[198]	train-auc:1.00000	test-auc:0.35000
[199]	train-auc:1.00000	test-auc:0.35000
User: P80, Avg. Accuracy: 0.4889, Avg. AUC: 0.5500, Avg. F1: 0.4667

Overall Metrics:
Average Accuracy: 0.6341
Average AUC: 0.6206
Average Macro F1 Score: 0.5115
Average Macro Precision: 0.5320
Average Macro Recall: 0.5464


/home/user/miniconda3/envs/sci-data/lib/python3.9/site-packages/xgboost/core.py:122: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  warnings.warn(


In [15]:
print(f"\nOverall Metrics:")
print(f"Average Accuracy: {np.mean(all_accs):.4f}")
print(f"Average AUC: {np.nanmean(all_aucs):.4f}")
print(f"Average Macro F1 Score: {np.mean(all_f1s):.4f}")
print(f"Average Macro Precision: {np.mean(all_precisions):.4f}")
print(f"Average Macro Recall: {np.mean(all_recalls):.4f}")
print(f"Average Positive F1 Score: {np.mean(all_f1p):.4f}")


Overall Metrics:
Average Accuracy: 0.6341
Average AUC: 0.6206
Average Macro F1 Score: 0.5115
Average Macro Precision: 0.5320
Average Macro Recall: 0.5464
Average Positive F1 Score: 0.3424
